# Рекоммендательные системы
### Дуботолкова Натали, 3530903/70301
### Вариант 7

2.1 Для заданного пользователя (совпадает с вашим номером в таблице) рассчитать оценки для всех фильмов, которые он не оценил.

In [27]:
import numpy as np
import pandas as pd
import json
import math as m
%matplotlib inline

In [28]:
data = pd.read_csv('data.csv', index_col=0)

In [29]:
data.head(7)

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,Movie 8,Movie 9,Movie 10,...,Movie 21,Movie 22,Movie 23,Movie 24,Movie 25,Movie 26,Movie 27,Movie 28,Movie 29,Movie 30
User 1,-1,4,1,5,-1,-1,-1,1,3,-1,...,2,4,-1,-1,3,4,-1,2,4,1
User 2,4,5,2,3,3,4,-1,5,3,2,...,-1,2,-1,3,1,5,-1,5,2,2
User 3,4,2,5,4,1,5,-1,3,5,2,...,4,-1,5,4,5,1,2,1,-1,3
User 4,5,-1,2,-1,-1,2,4,3,2,3,...,-1,2,2,2,2,5,3,2,5,5
User 5,4,3,3,3,5,3,3,4,5,2,...,4,5,2,-1,3,5,1,-1,5,-1
User 6,2,-1,3,3,3,3,4,2,-1,1,...,5,-1,1,3,-1,4,4,5,1,2
User 7,4,1,4,4,1,4,5,2,2,-1,...,2,5,1,3,4,-1,4,3,1,4


In [30]:
k = 4
my_user = 6

In [31]:
def sim(u,v):
    sum_uv, sum_u, sum_v = 0, 0, 0
    for i in range (len(u)):
        if u[i]>0:
            sum_u += u[i]**2
            if v[i]>0:
                sum_v += v[i]**2
                sum_uv += v[i]*u[i]
    return round((sum_uv/(m.sqrt(sum_u)*m.sqrt(sum_v))),3)
        

In [32]:
all_similarities = np.zeros((40, 40))

In [33]:
for i in range(data.shape[0]):
    listOfU = list(data.iloc[i])
    j = i+1
    for j in range(j, data.shape[0]):
        listOfV = list(data.iloc[j])
        all_similarities[i, j] = sim(listOfU, listOfV)
        all_similarities[j, i] = all_similarities[i, j]

In [34]:
all_similarities #матрица сходств всех пользователей

array([[0.   , 0.736, 0.63 , ..., 0.755, 0.635, 0.813],
       [0.736, 0.   , 0.7  , ..., 0.719, 0.628, 0.777],
       [0.63 , 0.7  , 0.   , ..., 0.762, 0.756, 0.627],
       ...,
       [0.755, 0.719, 0.762, ..., 0.   , 0.749, 0.806],
       [0.635, 0.628, 0.756, ..., 0.749, 0.   , 0.783],
       [0.813, 0.777, 0.627, ..., 0.806, 0.783, 0.   ]])

In [35]:
my_similarities = all_similarities[my_user] #матрица сходств для моего пользователя (7-го)

In [36]:
my_similarities

array([0.721, 0.778, 0.815, 0.736, 0.77 , 0.789, 0.   , 0.751, 0.714,
       0.724, 0.661, 0.786, 0.799, 0.734, 0.658, 0.839, 0.642, 0.811,
       0.783, 0.729, 0.837, 0.674, 0.69 , 0.772, 0.71 , 0.739, 0.759,
       0.76 , 0.763, 0.874, 0.818, 0.84 , 0.779, 0.734, 0.736, 0.791,
       0.694, 0.828, 0.738, 0.801])

In [37]:
my_LikeMinded = my_similarities.argsort()[-k:]
my_LikeMinded #массив индексов четырёх людей, у кого со мной наибольшее сходство

array([20, 15, 31, 29], dtype=int32)

In [38]:
#средняя оценка пользователя u
def average(u):
    sum_u = 0
    counter = 0
    for i in range (len(u)):
        if u[i]>0:
            counter += 1
            sum_u += u[i]
    return round((sum_u/counter),3)

In [39]:
# расчёт оценки на конкретный фильм
def r_ui(film):
    listOfU = list(data.iloc[my_user])
    r_u = average(listOfU)
    sum_up, sum_down = 0, 0
    for i in range(k):
        v = my_LikeMinded[i]
        listOfV = list(data.iloc[v])
        _sim = sim(listOfU, listOfV)
        sum_up += round((_sim*(data.iloc[v][film]-average(listOfV))), 3)
        sum_down += abs(_sim)
    return round((r_u + sum_up/sum_down),3)

In [40]:
listOfU = list(data.iloc[my_user])
listRes = {}
# находим индексы непросматриваемых фильмов и пытаемся предсказать оценку
for idx, num in enumerate(listOfU):
        if num<0:
            listRes['movie ' + str(idx+1) + ': '] = r_ui(idx)
listRes

{'movie 10: ': 1.539,
 'movie 12: ': 1.549,
 'movie 17: ': 1.799,
 'movie 18: ': 2.069,
 'movie 26: ': 3.807}

2. Порекомендовать заданному пользователю 1 фильм, который он посмотрел бы в выходной дома, если такой фильм есть в данных.

Я бы вообще не советовала ему ничего смотреть - у него слишком низкий средний балл, (~2,3) Так что высока вероятность, что предложенные фильмы ему тоже не понравятся.
Однако, попробуем посмотреть, какие фильмы смотрели по выходным дома те 4 человека, которые имеют с ним +- такой же вкус и выберем фильм с наибольшим рейтингом.

In [41]:
days = pd.read_csv("context_day.csv", index_col=0)
places = pd.read_csv("context_place.csv", index_col=0)

In [42]:
days.head(5)

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,Movie 8,Movie 9,Movie 10,...,Movie 21,Movie 22,Movie 23,Movie 24,Movie 25,Movie 26,Movie 27,Movie 28,Movie 29,Movie 30
User 1,-1,Thu,Tue,Mon,-1,-1,-1,Tue,Sun,-1,...,Thu,Thu,-1,-1,Sun,Sat,-1,Wed,Wed,Thu
User 2,Mon,Wed,Mon,Sat,Sat,Thu,-1,Mon,Fri,Thu,...,-1,Sun,-1,Mon,Mon,Fri,-1,Sun,Tue,Wed
User 3,Tue,Wed,Sat,Tue,Sat,Mon,-1,Thu,Wed,Thu,...,Tue,-1,Wed,Thu,Tue,Thu,Wed,Wed,-1,Sun
User 4,Mon,-1,Wed,-1,-1,Sun,Sat,Mon,Mon,Wed,...,-1,Sun,Fri,Sat,Thu,Sat,Fri,Fri,Sat,Sun
User 5,Thu,Fri,Wed,Fri,Mon,Thu,Sat,Tue,Thu,Fri,...,Fri,Fri,Mon,-1,Wed,Mon,Sat,-1,Wed,-1


In [43]:
places.head(5)

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,Movie 8,Movie 9,Movie 10,...,Movie 21,Movie 22,Movie 23,Movie 24,Movie 25,Movie 26,Movie 27,Movie 28,Movie 29,Movie 30
,,,,,,,,,,,,,,,,,,,,,
User 1,-1,h,h,c,-1,-1,-1,v,h,-1,...,h,c,-1,-1,c,c,-1,v,h,c
User 2,v,v,v,v,h,c,-1,v,h,c,...,-1,c,-1,c,h,v,-1,v,v,c
User 3,v,h,c,c,c,c,-1,h,c,v,...,v,-1,v,v,h,h,h,v,-1,v
User 4,v,-1,h,-1,-1,h,v,v,c,h,...,-1,v,h,h,c,v,c,v,v,c
User 5,c,c,c,c,v,h,v,v,c,h,...,v,c,v,-1,h,c,v,-1,h,-1


In [44]:
#находим фильмы (их индексы), которые пользователь ещё не смотрел
duser = data.iloc[my_user]
user_films = []
for index, i in enumerate(duser):
    if i<0:
        user_films.append(index)
user_films    

[9, 11, 16, 17, 25]

In [45]:
# нашла фильмы, которые наша четверка смотрела по выходным
allFilmsOnWeekend = []
for user in my_LikeMinded:
    for idx,el in enumerate(days.iloc[user]):
        if el.strip() == 'Sat' or el.strip() == 'Sun':
            allFilmsOnWeekend.append(idx+1)
auxiliaryList = list(set(allFilmsOnWeekend))
auxiliaryList

[1, 2, 4, 5, 6, 7, 8, 10, 12, 13, 16, 17, 19, 21, 25, 26, 27, 28, 29, 30]

In [46]:
#нашли совпадения в этих двух списках
films = list(set(user_films) & set(auxiliaryList))
print(films)

[16, 17, 25]


In [47]:
# нашла фильмы, которые наша четверка смотрела дома
allFilmsAtHome = []
for user in my_LikeMinded:
    for idx,el in enumerate(places.iloc[user]):
        if el.strip() == 'h':
            allFilmsAtHome.append(idx+1)   
auxiliaryList = list(set(allFilmsOnWeekend))
auxiliaryList

[1, 2, 4, 5, 6, 7, 8, 10, 12, 13, 16, 17, 19, 21, 25, 26, 27, 28, 29, 30]

In [48]:
#нашли совпадения в этих двух списках
films = list(set(films) & set(auxiliaryList))
print(films)

[16, 17, 25]


In [49]:
# посчитала рейтинг каждого фильма, среди единомышленников
count = 0
raiting_film = {}
temp = 0

for i in films:
    for j in my_LikeMinded:
        raiting = data.iloc[j][i]
#         print(i, raiting)
        if raiting > 0:
            temp += 1
            count+= raiting
    raiting_film[i] = round(count / temp, 3)

raiting_film

{16: 2.0, 17: 2.571, 25: 3.091}

In [50]:
#нашла максимальное значение среди всех фильмов
max_val = max(raiting_film.values())
recom_film = {f'Movie {k}:':v for k, v in raiting_film.items() if v == max_val}
recom_film

{'Movie 25:': 3.091}

In [51]:
result = {"user": 7, "1": listRes, "2": recom_film}
with open('result.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)